In [141]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pickle
import zlib

In [142]:
DEBUG = True

# OrthoGroups to MSA 
+ load table with the proteins - orthogrous distribution 
+ filter the core orthogroups 
+ load proteins .pkl database
+ get the proteins and the genome positions corresponding to the core OGs

In [143]:
if DEBUG:
    ids = 'rso_test' #'korea2' #'rso_test'  #

    core_ogs = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_'+ids+'/Species_Tree/Orthogroups_for_concatenated_alignment.txt'
    ogs2prot = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_'+ids+'/Orthogroups/Orthogroups.tsv'
    prot_db = pd.read_pickle('../'+ids+'_outputs/GENOME_PROTEIN_DB_'+ids+'.pkl')
    path_p_aln = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_'+ids+'/MultipleSequenceAlignments/'

    msa_out_fasta = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/'+ids+'_concat.fasta'
    msa_out_phylip = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/'+ids+'_concat.nexus'
    msa_out_xmfa_path = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/'+ids+'_concat.xmfa'
    aln2acc = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/outputs/MSA_ACC_DB_'+ids+'.pkl'
else:
    core_ogs = snakemake.input["core_ogs"]
    ogs2prot = snakemake.input["ogs2prot"]
    prot_db = pd.read_pickle(snakemake.input["prot_db"])
    path_p_aln = snakemake.input["path2aln"]

    msa_out_fasta = snakemake.output["msa_fasta"]
    msa_out_phylip = snakemake.output["msa_phylip"]
    msa_out_xmfa_path = snakemake.output["msa_xmfa"]
    aln2acc = snakemake.output["aln2acc"]

In [144]:
strainsNames2fileNames = {}
for c in prot_db.columns:
    strainsNames2fileNames[c.split('$')[1]] = c.split('$')[0]
strainsNames2fileNames

{'GCF_013306935.1_ASM1330693v1': 'FJAT1303.F50',
 'GCF_013306235.1_ASM1330623v1': 'FJAT445.F50',
 'GCF_013306435.1_ASM1330643v1': 'FJAT15304.F6',
 'GCF_001887535.1_ASM188753v1': 'FJAT-1458',
 'GCF_013306335.1_ASM1330633v1': 'FJAT15353.F50',
 'GCF_013375735.1_ASM1337573v1': 'FJAT91-F8'}

In [145]:
prot_db = prot_db.rename(columns=lambda x: str(x).split('$')[0])
prot_db

,FJAT1303.F50,FJAT445.F50,FJAT15304.F6,FJAT-1458,FJAT15353.F50,FJAT91-F8
WP_071507848.1,"{'chr': 'NZ_CP052126.1', 'nucl': b'x\x9cE\x94\...",NaN,NaN,NaN,"{'chr': 'NZ_CP052086.1', 'nucl': b'x\x9cE\x94\...",NaN
WP_013210791.1,"{'chr': 'NZ_CP052126.1', 'nucl': b'x\x9cM\x93\...",NaN,NaN,NaN,"{'chr': 'NZ_CP052086.1', 'nucl': b'x\x9cM\x93\...",NaN
WP_173941386.1,"{'chr': 'NZ_CP052126.1', 'nucl': b'x\x9cM\x96\...",NaN,NaN,NaN,"{'chr': 'NZ_CP052086.1', 'nucl': b'x\x9cM\x96\...",NaN
WP_071615401.1,"{'chr': 'NZ_CP052126.1', 'nucl': b'x\x9c5\x92\...",NaN,"{'chr': 'NZ_CP052096.1', 'nucl': b'x\x9c5\x92\...",NaN,"{'chr': 'NZ_CP052086.1', 'nucl': b'x\x9c5\x92\...",NaN
WP_071615399.1,"{'chr': 'NZ_CP052126.1', 'nucl': b'x\x9cE\x92Q...",NaN,"{'chr': 'NZ_CP052096.1', 'nucl': b'x\x9cE\x92Q...",NaN,"{'chr': 'NZ_CP052086.1', 'nucl': b'x\x9cE\x92Q...",NaN
...,...,...,...,...,...,...
WP_016726112.1,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP056086.1', 'nucl': b'x\x9ce\x94\..."
WP_028861767.1,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP056086.1', 'nucl': b""x\x9ceS\xc9..."
WP_023470131.1,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP056086.1', 'nucl': b""x\x9cMT\xdb..."
WP_043885778.1,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP056086.1', 'nucl': b'x\x9cM\x94\..."


In [146]:
# get core ogs
f = open(core_ogs,"r")
ogs = []
for l in f:
    ogs.append(l.replace('\n',''))
f.close()
##print(ogs)

In [147]:
# load roteins ids from the OGs table 
# keep only the core OGs

tab_prot = pd.read_csv(ogs2prot, sep='\t', index_col=0)  
tab_prot = tab_prot.loc[ogs]
tab_prot = tab_prot.rename(columns=lambda x: str(x).replace('_protein','')) # ncbi file names correction
tab_prot = tab_prot.rename(columns=strainsNames2fileNames)
tab_prot

,FJAT-1458,FJAT445.F50,FJAT15353.F50,FJAT15304.F6,FJAT1303.F50,FJAT91-F8
Orthogroup,,,,,,
OG0000225,WP_003261911.1,WP_003261911.1,WP_003261911.1,WP_003261911.1,WP_003261911.1,WP_003261911.1
OG0000226,WP_003261929.1,WP_003261929.1,WP_003261929.1,WP_003261929.1,WP_003261929.1,WP_003261929.1
OG0000227,WP_003261939.1,WP_003261939.1,WP_003261939.1,WP_003261939.1,WP_003261939.1,WP_003261939.1
OG0000228,WP_003261941.1,WP_003261941.1,WP_003261941.1,WP_003261941.1,WP_003261941.1,WP_003261941.1
OG0000229,WP_003262123.1,WP_003262123.1,WP_003262123.1,WP_003262123.1,WP_003262123.1,WP_003262123.1
...,...,...,...,...,...,...
OG0004089,WP_155772886.1,WP_148668905.1,WP_148668915.1,WP_148668915.1,WP_148668915.1,WP_155772886.1
OG0004091,WP_155773184.1,WP_173940865.1,WP_173941733.1,WP_119889930.1,WP_173952290.1,WP_086706436.1
OG0004092,WP_155773186.1,WP_144061950.1,WP_119417730.1,WP_119889926.1,WP_119417730.1,WP_038938529.1


In [148]:
genomes_for_og = list(tab_prot.columns) 

#for x in ['pe1', 'pe3', 'pe27', 'pe57', 'pe39']:
#    genomes_for_og.remove(x)
#genomes_for_og

# Get Nucleotide sequences + corresponding positions in genome

# Load single proteins alignments and extract gaps

In [149]:
gaps2tab = pd.DataFrame(columns=tab_prot.columns, index=tab_prot.index)

len_counter = 0

for og in ogs:
    pmsa = path_p_aln + '/' +og + '.fa'
    genome_ids = []
    #if len_counter >= 395140:
      #  print(og, '-----')
        #len_counter = 0
            
    for record in SeqIO.parse(pmsa, "fasta"):  
        #print(pmsa)
        
        # prokka/ncbi genomes header structure 
        header = record.id.split('_')
        protein_id = header[-2]+'_'+header[-1]
        genome_id = strainsNames2fileNames[record.id.replace(protein_id, '').replace('protein_', '')[:-1].replace('_1_', '.1_')]
        #genome_ids.append(genome_id)
        prot_db.loc[protein_id, genome_id]['og'] = og
        
        #print(pmsa)
        #print(protein_id, genome_id)
        
        # GAPS PROFILE of aa alignment  
        gaps_bool = []
        
        #if og == 'OG0000732' and genome_id == 'to53':
            #print('---', len(record.seq) )
        
        for aa in record.seq:
            if aa == '-':
                gaps_bool.append(1) 
                gaps_bool.append(1) 
                gaps_bool.append(1) 
            else:
                gaps_bool.append(0) 
                gaps_bool.append(0) 
                gaps_bool.append(0)
        
        
        #if genome_id == 'to53':
        #    print(gaps_bool)
        #    print('to53',record.seq)
        #if genome_id == 'to7':
        #    print(gaps_bool)
        #    print('to7', record.seq)
        prot_db.loc[protein_id, genome_id]['aln_gaps'] = gaps_bool
        prot_db.loc[protein_id, genome_id]['og'] = og
        gaps2tab.loc[og][genome_id] = gaps_bool
        
        #if len(record.seq)*3 != len(prot_db.loc[protein_id, genome_id]['nucl']):
            #print(protein_id, genome_id)
            #print('ffffff',len(record.seq)*3, len(prot_db.loc[protein_id, genome_id]['nucl']))
    #print(og, genome_ids)
gaps2tab

,FJAT-1458,FJAT445.F50,FJAT15353.F50,FJAT15304.F6,FJAT1303.F50,FJAT91-F8
Orthogroup,,,,,,
OG0000225,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000226,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000227,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000228,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000229,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
OG0004089,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0004091,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0004092,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


import ast
d = ast.literal_eval(prot_db.loc[protein_id, genome_id])
d
d['og'] = og
prot_db.loc[protein_id, genome_id] = d

In [150]:
ogs_to_drop_from_core = []
for g in genomes_for_og:
    for og in ogs:
        
        msa_l = len(prot_db.loc[tab_prot.loc[og, g], g]['aln_gaps'])
        seq_l = len(zlib.decompress(prot_db.loc[tab_prot.loc[og, g], g]['nucl']).decode())
        gap_l = gaps2tab.loc[og, g].count(1)
        
        if seq_l + gap_l != msa_l:
            print('Length error in:', g, og, ': MSA length:', msa_l, 'protein length:' ,seq_l)
            ogs_to_drop_from_core.append(og)
            
tab_prot = tab_prot.drop(ogs_to_drop_from_core)
gaps2tab = gaps2tab.drop(ogs_to_drop_from_core)
ogs = [i for i in ogs if i not in ogs_to_drop_from_core]

In [153]:
import gzip
s_in = b"Lots of content here"
s_out = gzip.compress(s_in)
s_out

b'\x1f\x8b\x08\x00\x0f|Eb\x02\xff\xf3\xc9/)V\xc8OSH\xce\xcf+I\xcd+Q\xc8H-J\x05\x00~u \x8e\x14\x00\x00\x00'

In [154]:
prot_db.loc[tab_prot.loc[og, g], g]['nucl']

b'x\x9cUU\xd1\x95\xc30\x08\x9bM\x8f\x0f\x16\xd0\xfe\xb3\x1cH\xc2\xe9\xb5i\x9b\xd8\x18\x84\x10\x14lT7\xbaP\x98\xefj\xbd\xd1l\xd2\x0f\xb3\x8bn\xdb`\x96\xc7h~\xc6d\xb6lRZ\xdc\xbby\x9e\xaf\xb1\x9d\xcf\\\xbbX\xd4G\xd6@\xac[A\xb8\x8b\x8a\xb6\xab\xe7d.\x99e\x8f\xf2Sq8[\x95X\xb21f?\xb6P\xb5\x8f\xb2}\xa4\xb2%`\x86\xa4\xe8p$\xa7/L\x1co\xba\x0eAB8\xa5\xf1<x!HuG\x0c\x06\xa2\xae\xef\xbe\xcfsQ\xfcP\x80X\xc6\xef\xb3\xb8\x9b\xcb\xc59(G\x88y\xca&\xe0\x06\xb9\xec\x10\xa6g\x97\xa9\x97L\x94\xcf&\xe0D\xa9d\xb8\x80\x8d!\xc4\x89\x9c\x8d\xe0\xf4\xb5\xe6\n\x82)\x04^\xb1V\x10\xaeS\xd7\xab\xa6v\xb9\x07\xc3SE\'\xa5\x83\x82Q\xf6\x9b\xa0Tz\x8b_\xebA\xb2 \xe6\xb5\xebx\t2J\xa0\xf0-\xa5N\xb6we\xaeT\x9c\x16C\\-\xe1b\xff\x83j\x0fR)\x92^\xd8\x95`8\x8b\xe4\x13\xdb\x9a\xd1\xd2\x15\xf3\xfc\xd5\x0f\xda\t\xa45\xa4~\xe0\x08w\x95G\xa1\x97\x85\x99\x84v\xbd\x82J\ra\xc6\x19W\x93|\xc0[\x90LD\xe3\x92B,\xbcT>]\xc3p\xbe\r*f \xe4i+\xb2O\xb6\xb4\x97\xc3\xdca@\xa9\xb8\xd0*\n\x94\xf5RH\x1fa\xba\xbf~b=\x95"\xe5\x88\x8c=\x03\xeax\xb8\x1et\x

# Genome-based solution
+ for each genome grab the cols of the core OG table
+ concatenate the corresponding nucleotides from the GENOME_PROTEIN_DB

## TODO length of nucleotides is larger than length of translation*3 ?!

In [155]:
g2nucl = {}
g2pos = {}
g2chr = {}
g2strand = {}
g2acc = {}
g2gaps = {}
g2og = {}

for g in genomes_for_og:
    print('Genome', g)
    
    # get corresponding nucleotide positions 
    g2pos[g] = []
    # get corresponding chr ids 
    g2chr[g] = []
    # get corresponding strands
    g2strand[g] = []
    # get protein accession for each genome position
    g2acc[g] = []
    # get gaps profile
    g2gaps[g] = []
    # debug
    g2og[g] = []
    
    # get nucleotide sequences for all the core og proteins per genome 
    nucl_per_genome = []
    #print(prot_db.loc[tab_prot[g].values, g])
    
    for i in prot_db.loc[tab_prot[g].values, g]: 
        
        nucl_per_genome.append(zlib.decompress(i['nucl']).decode())
        #g2pos[g].extend(i['location'])
        #print('----',g, )
        g2og[g].append(i['og'])
        
        for counter in range(len(i['aln_gaps'])):
            #g2chr[g].append(i['chr'])
            #g2strand[g].append(i['location'].strand)
            g2acc[g].append(i['acc'])
        g2gaps[g].extend(i['aln_gaps'])
    
    nucl_per_genome = "".join(nucl_per_genome)
    g2nucl[g] = nucl_per_genome
    print('Exported sequence length NO GAPS', len(nucl_per_genome))
    print('Exported alignment WITH GAPS', len(g2gaps[g]))
    print('Exported protein accessions', len(g2acc[g]))
    print('Exported numeric positions', len(g2pos[g]))
    print('Exported chromosomal positions', len(g2chr[g]))
    print('Exported strands orientations', len(g2strand[g]))
    print()
    
    ####### debug 
    #g2nucl[g] = g2nucl[g][:395150]
    #g2gaps[g] = g2gaps[g][:395150]
    #g2acc[g] = g2acc[g][:395150]
    

Genome FJAT-1458
Exported sequence length NO GAPS 3816252
Exported alignment WITH GAPS 3825198
Exported protein accessions 3825198
Exported numeric positions 0
Exported chromosomal positions 0
Exported strands orientations 0

Genome FJAT445.F50
Exported sequence length NO GAPS 3816393
Exported alignment WITH GAPS 3825198
Exported protein accessions 3825198
Exported numeric positions 0
Exported chromosomal positions 0
Exported strands orientations 0

Genome FJAT15353.F50
Exported sequence length NO GAPS 3813270
Exported alignment WITH GAPS 3825198
Exported protein accessions 3825198
Exported numeric positions 0
Exported chromosomal positions 0
Exported strands orientations 0

Genome FJAT15304.F6
Exported sequence length NO GAPS 3814989
Exported alignment WITH GAPS 3825198
Exported protein accessions 3825198
Exported numeric positions 0
Exported chromosomal positions 0
Exported strands orientations 0

Genome FJAT1303.F50
Exported sequence length NO GAPS 3812958
Exported alignment WITH GA

# MSA repeats problem 

some proteins sequence of the OrthoFinder may have long repeat sequences, which are not present in the orthologs. In the resulting MSA produced by OrthoFinder the repeated sequences are skipped:

e.g. 
+ original input: <br>
\> p1 <br>
ABCXYZ <br>
\> p2 <br>
ABCABCXYZ <br>
+ MSA output:<br>
\> p1 <br>
ABCXYZ <br>
\> p2 <br>
ABCXYZ <br>

This results in large shifts in the final nucleotides MSA <br>

Examples from to53

OG: OG0000732; gene: DEOEPAP_01560
MNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK
VNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHT                                                                                                                                                                                                                                                                                                                                       VVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK

OG: OG0002212; gene: GDEOEPAP_02088
MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQPLQPPG
MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQ   PPG

OG: OG0003692; gene: GDEOEPAP_04673
MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGG  TGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE
MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGGGGTGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE

In [156]:
import difflib

s_nucl ="MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQPLQPPG"
s_msa = "MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQPPG"

s_msa = "MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGGTGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE"
s_nucl =  "MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGGGGTGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE"

s_nucl ="MNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK"
s_msa = "VNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK"

#s1-s2
output_list = [li for li in difflib.ndiff(s_nucl, s_msa) if li[0] != ' ']
print(output_list)

repeat_seq = ''.join([i[-1] for i in output_list if i[0] == '-'])
print('rep:', repeat_seq)

# count diff in nucl string 
print(s_nucl.count(repeat_seq[1:]))
print(s_msa.count(repeat_seq[1:]))

['- M', '- N', '- S', '- K', '- T', '- D', '- R', '- P', '- A', '- E', '- G', '- A', '- G', '- A', '- T', '- A', '- D', '- R', '- A', '- A', '- R', '- E', '- A', '- A', '- V', '- A', '- R', '- A', '- L', '- E', '- C', '- L', '- A', '- D', '- H', '- A', '- G', '- D', '- H', '- Q', '- T', '- L', '- T', '- G', '- E', '- P', '- L', '- V', '- S', '- H', '- A', '- R', '- G', '- T', '- V', '- A', '- I', '- L', '- E', '- G', '- L', '- R', '- V', '- D', '- A', '- P', '- S', '- L', '- Q', '- A', '- A', '- A', '- L', '- F', '- L', '- L', '- P', '- I', '- L', '- A', '- T', '- D', '- N', '- E', '- K', '- A', '- I', '- E', '- P', '- A', '- F', '- G', '- G', '- E', '- V', '- A', '- R', '- L', '- V', '- H', '- D', '- V', '- R', '- Q', '- L', '- L', '- R', '- I', '- G', '- A', '- I', '- A', '- G', '- M', '- A', '- S', '- P', '- A', '- E', '- A', '- G', '- V', '- T', '- R', '- K', '- N', '- E', '- A', '- E', '- A', '- R', '- R', '- A', '- Q', '- V', '- E', '- A', '- L', '- R', '- K', '- M', '- L', '- L'

In [157]:
# SAVE PROTEIN ACC FOR MAPPING

with open(aln2acc, 'wb') as handle:
    pickle.dump(g2acc, handle, protocol=pickle.HIGHEST_PROTOCOL)

# replace ambiguous bases with gaps!

In [158]:
for g in g2nucl:
    atcg = {'A', 'C', 'T', 'G'}
    ambig = set(g2nucl[g]).difference(atcg)
    
    print("before", set(g2nucl[g]))
    for b in ambig:
        g2nucl[g] = g2nucl[g].replace(b, '-')
    print("after ",set(g2nucl[g]))

before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}


In [159]:
#str(prot_db.loc["WP_097388881.1","GCF_002501565.1_ASM250156v1_protein"]['nucl'])
#for v in prot_db['GCF_001587135.1_ASM158713v1_protein'].values:
#    print(v)

# Export MSA

In [160]:
msa_fasta = open(msa_out_fasta, "w")
for g in g2nucl:
    print(g, len(g2nucl[g]))
    msa_fasta.write('>'+g+'\n') ##+g.split('_')[2][:10]+'\n')
    
    i = 0
    for gap in g2gaps[g]:
        if gap:
            msa_fasta.write('-')
        else:
            msa_fasta.write(g2nucl[g][i])
            i = i + 1
    msa_fasta.write('\n')
msa_fasta.close()

FJAT-1458 3816252
FJAT445.F50 3816393
FJAT15353.F50 3813270
FJAT15304.F6 3814989
FJAT1303.F50 3812958
FJAT91-F8 3815583


# Convert to Phylip

In [161]:
#SeqIO.convert(msa_out_fasta, "fasta", msa_out_phylip, "nexus")

nexus = open(msa_out_phylip, 'w')
nexus.write('#NEXUS\n')
nexus.write('begin data;\n')
nexus.write('dimensions ntax='+str(len(genomes_for_og))+' nchar='+str(len(g2gaps[g]))+' ;\n')
nexus.write('format datatype=dna ;\n')
nexus.write('matrix\n')

for record in SeqIO.parse(msa_out_fasta, "fasta"):
    nexus.write(record.description+'\t'+str(record.seq)+'\n')
    
nexus.write(';\n')
nexus.write('end;\n')

nexus.close()
#SeqIO.write(annotated_records, msa_out_phylip, "nexus")


# Export XMFA

In [162]:
msa_out_xmfa = open(msa_out_xmfa_path, 'w')

for og in ogs:
    pmsa = path_p_aln + '/' +og + '.fa'
    for record in SeqIO.parse(pmsa, "fasta"):
        #print(record.id)
        #g_id = record.id.split('_WP')[0].replace('_1_', '.1_')
        #p_id = record.id.split('_protein_')[-1]
        # prokka changes 
        ##g_id = record.id.split('_')[0]
        ##p_id = record.id.replace(record.id.split('_')[0]+'_', '')
        
        # prokka/ncbi genomes header structure 
        header = record.id.split('_')
        p_id = header[-2]+'_'+header[-1]
        g_id = strainsNames2fileNames[record.id.replace(p_id, '').replace('protein_', '')[:-1].replace('_1_', '.1_')]       
        #print(g_id, p_id)
                                           
        nns = zlib.decompress(prot_db.loc[p_id, g_id]['nucl']).decode() 
        #msa_out_xmfa.write('>'+record.id.split('_')[3][:10]+'\n')
        #prokka
        msa_out_xmfa.write('>'+g_id+'\n')
        
        atcg = {'A', 'T', 'G', 'C','-'}
        ambig = set(nns).difference(atcg)
        #print(ambig)
        for b in ambig:
            nns = str(nns).replace(b, '-')
        #print('aaaaaa',  nns)
        
        gaps_v = gaps2tab.loc[og, g_id]
        i = 0
        for gap in gaps_v:
            if gap:
                msa_out_xmfa.write('-')
            else:
                msa_out_xmfa.write(nns[i])
                i = i + 1
        msa_out_xmfa.write('\n')
    msa_out_xmfa.write('=' + '\n')
msa_out_xmfa.close()

# Test the position mapping
+ grab sequences from the original gb files using g2pos, g2strand and g2chr
+ load exported core OG nucleotide fasta file
+ compare the sequences 

(note: reverse compliment!)

In [163]:
if DEBUG:
    # for the compliment strand mapping
    from Bio.Seq import Seq
    Seq("A").complement()

In [164]:
if DEBUG:
    import os
    gbks = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/inputs/panX/'+ids
    ccc=0

    test_fasta = {}
    for entry in os.scandir(gbks):
        #if ccc>0:
        #    break
        ccc = ccc+1
        if (entry.path.endswith(".gbk") and entry.is_file()):
            genome = os.path.basename(entry).replace('.gbk', '').replace('_genomic', '')
            genome = strainsNames2fileNames[genome]
            test_fasta[genome] = []
            print('Genome:', genome)

            # map chr id to the sequence
            chr_seq = {}
            for record in SeqIO.parse(entry.path, "gb"):
                print('Chr',record.id, 'with', len(record.features), 'features')
                chr_seq[record.id] = record.seq

            # get corresponding mapping (from above)
            mapping_chr = g2chr[genome]
            mapping_pos = g2pos[genome]
            mapping_str = g2strand[genome]
            
            print(g2chr[genome][:10])
            print(g2pos[genome][:10])
            print(g2strand[genome][:10])
            
            print(len(mapping_chr), len(mapping_pos))
            assert(len(mapping_chr) == len(mapping_pos))

            # iterate through the original positions 
            for i in range(len(mapping_chr)):
                if mapping_str[i] == 1:
                    test_fasta[genome].append(chr_seq[mapping_chr[i]][mapping_pos[i]])
                else:
                    compl = str(Seq(chr_seq[mapping_chr[i]][mapping_pos[i]]).complement())
                    test_fasta[genome].append(compl)
            test_fasta[genome] = "".join(test_fasta[genome])
            print("Core nucl exported:", len(test_fasta[genome]))

            print()
            

Genome: FJAT1303.F50
Chr NZ_CP052126.1 with 7264 features
Chr NZ_CP052127.1 with 3123 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: FJAT445.F50
Chr NZ_CP052076.1 with 6856 features
Chr NZ_CP052077.1 with 3177 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: FJAT15304.F6
Chr NZ_CP052096.1 with 6822 features
Chr NZ_CP052097.1 with 3221 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: FJAT-1458
Chr NZ_CP016554.1 with 7588 features
Chr NZ_CP016555.1 with 3271 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: FJAT15353.F50
Chr NZ_CP052086.1 with 7262 features
Chr NZ_CP052087.1 with 3127 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: FJAT91-F8
Chr NZ_CP056085.1 with 7282 features
Chr NZ_CP056086.1 with 3165 features
[]
[]
[]
0 0
Core nucl exported: 0



In [165]:
if DEBUG:

    for record in SeqIO.parse(msa_out_fasta, "fasta"):
        print(record.id, len(record.seq))
        print('test length', len(record.seq))
        print('Test:', test_fasta[record.id] == str(record.seq))
        print()


FJAT-1458 3825198
test length 3825198
Test: False

FJAT445.F50 3825198
test length 3825198
Test: False

FJAT15353.F50 3825198
test length 3825198
Test: False

FJAT15304.F6 3825198
test length 3825198
Test: False

FJAT1303.F50 3825198
test length 3825198
Test: False

FJAT91-F8 3825198
test length 3825198
Test: False



f = "/Users/devseeva/Desktop/work/pan-genome-visualization/data/ncbi-rso/metainfo.tsv"
x = pd.read_csv(f, sep='\t')  
phys = []
for a in x["accession"]:
    phys.append(a.split(".1_")[1][:10])
x["phy_id"] = phys
x.to_csv("/Users/devseeva/Desktop/work/sm_workflow/snakefiles/inputs/phy4_test_metadata/phy4_test_cleaned_data.csv", index=False)